# Laboratorio 1 - Preparación y Visualización de Datos

## Grupo 1 - Paula Hípola Gómez, José Ignacio Navas Sanz y Belén Ortega Pérez

## Entregable 3 - Principal Component Analysis (PCA)

### 3.1 – (0.5 puntos) Estandarización y normalización de datos. 
#### Para evitar que las diferencias de rango en los datos supongan un problema a la hora de procesar la información, el primer paso es estandarizar y normalizar los datos. Usad el método StandardScaler de la librería scikit-learn.

In [35]:
%config IPCompleter.greedy=True

#Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

#Importamos el CSV del FIFA
df_fifa = pd.read_csv('fifa19.csv')

#Sacamos por pantalla los primeros datos para ver el dataset
df_fifa.head()


,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,...,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,...,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,...,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,...,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,4,192985,K. De Bruyne,27,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,92,Manchester City,...,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M


In [36]:
# Para estandarizar y normalizar solo nos valen los valores numericos por lo que eliminamos todos los que sean string
df = df_fifa.select_dtypes(exclude=['object'])
df.head()

,Unnamed: 0,ID,Age,Overall,Potential,Special,International Reputation,Weak Foot,Skill Moves,Jersey Number,...,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes
0,0,158023,31,94,94,2202,5.0,4.0,4.0,10.0,...,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0
1,1,20801,33,94,94,2228,5.0,4.0,5.0,7.0,...,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0
2,2,190871,26,92,93,2143,5.0,5.0,5.0,10.0,...,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0
3,3,193080,27,91,93,1471,4.0,3.0,1.0,1.0,...,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0
4,4,192985,27,91,92,2281,4.0,5.0,4.0,7.0,...,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0


In [37]:
#Limpieza de datos
print('Tamaño sin limpiar',df.shape[0])
#Eliminacion de nulos
df = df.dropna()
print('Tamaño sin nulos',df.shape[0])

#Eliminacion de duplicados
df = df.drop_duplicates()
print('Tamaño sin duplicados',df.shape[0])

#Reseteamos el indice de las filas
df = df.reset_index()

#Imprimimos los datos
df.head()

Tamaño sin limpiar 18207
Tamaño sin nulos 18147
Tamaño sin duplicados 18147


,index,Unnamed: 0,ID,Age,Overall,Potential,Special,International Reputation,Weak Foot,Skill Moves,...,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes
0,0,0,158023,31,94,94,2202,5.0,4.0,4.0,...,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0
1,1,1,20801,33,94,94,2228,5.0,4.0,5.0,...,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0
2,2,2,190871,26,92,93,2143,5.0,5.0,5.0,...,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0
3,3,3,193080,27,91,93,1471,4.0,3.0,1.0,...,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0
4,4,4,192985,27,91,92,2281,4.0,5.0,4.0,...,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0


In [42]:
#Pasamos a estandarizar y normalizar los datos con la libreria StandardScaler
scaler = StandardScaler()
df_transformed = scaler.fit_transform(df)
print(df_transformed)

[[-1.72872243 -1.72872243 -1.87713614 ... -0.07471326 -0.14026605
  -0.48515932]
 [-1.72853223 -1.72853223 -6.45585886 ... -0.07471326 -0.14026605
  -0.31809379]
 [-1.72834204 -1.72834204 -0.78108824 ... -0.07471326 -0.08156876
  -0.31809379]
 ...
 [ 1.73357634  1.73357634  0.91286766 ... -0.37767658 -0.60984437
  -0.20671677]
 [ 1.73376654  1.73376654  1.06735809 ... -0.13530592 -0.49244979
  -0.42947081]
 [ 1.73395673  1.73395673  1.06739146 ... -0.43826925 -0.25766063
  -0.42947081]]


### 3.2 – (2 puntos) Autovalores y Autovectores
#### El siguiente paso es obtener los autovalores y autovectores que nos permitan caracterizar el espacio de llegada de la proyección donde vamos a reducir la dimensión de nuestros datos, estudiando cuántos componentes (dimensiones) necesitamos para representar los datos iniciales. Para esto, primero hay que obtener la matriz de covarianza utilizando el método cov de la librería numpy, y luego obtener los autovalores/autovectores de la matriz utilizando el método linalg.eig también de la librería numpy. Al final de este apartado, tienes que obtener un DataFrame con el porcentaje de varianza y el porcentaje acumulado por cada componente. Además, tienes que explicar qué quieren decir estos datos y cómo se relacionan con los datos.

In [ ]:
#Ya tenemos calculada la matriz con los datos normalizados, por lo que procedemos a obtener la matriz de covarianza


### 3.3 – (1 punto) Representar gráficamente los elementos del dataset
#### Finalmente, queremos representar gráficamente los elementos de nuestro dataset utilizando la descomposición que hemos calculado en el apartado 3.2. Obtén un diagrama de dispersión en 2 dimensiones y comenta/interpreta el resultado. Es necesario que el diagrama contenga toda la información necesaria. Además, tienes que interpretar la información que proporciona el eje X y el eje Y. Para terminar, elige al menos 4 individuos ¿qué puedes decir de ellos después del análisis que has realizado?

### 3.4- (0.5 punto) Repite los pasos 3.2 y 3.3 utilizando las funcionalidades disponibles en la librería scikitlearn.
#### Compara los resultados y coméntalos. ¿Hay alguna diferencia notable entre las dos implementaciones?